In [1]:
import sys
import numpy as np
import pickle
import h5py
import bz2
from numpy.random import poisson
import math
import re

# Convert pileup ascii to numpy

In [ ]:
def dat2arrays(filenames_in, filename_out):  
    number = r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?"
    vector = r"\[(%s),(%s),(%s),(%s)\]" % (number, number, number, number)
    
    n_events = 0
    n_particles = 0
    
    with bz2.BZ2File(filename_out, "w") as output_file:
        for filename in filenames_in:
            with open(filename, "r") as f:
                for line in f:
                    if line.startswith("HADS: "):
                        n_events += 1
                        e = []
                        for p in re.findall(vector, line):
                            n_particles += 1
                            e.append([float(p[0]), float(p[1]), float(p[2]), float(p[3])])
                    
                        e = np.array(e, dtype=np.float16)
                        pickle.dump(e, output_file, protocol=2)

                        if n_events % 10000 == 0:
                            print(n_events, 'events with mean of', n_particles / n_events, 'particles done')

In [ ]:
# Go!
folder = '/scratch/jb6504/data'
#folder = '../data'

filenames_in = [folder + '/pileup/pileup' + str(i) + '_ascii.dat' for i in range(1,5)]
filename_out = folder + '/pileup/pileup.dat.tar.bz2'

dat2arrays(filenames_in, filename_out)

# Jets, particle level

In [10]:
def add_pileup_jet_particle(hard_filename, output_filename,
                            pileup_file, n_expected, use_poisson=True):
    
    ''' Adds pileup to a particle-level event file. '''

    print('')
    print('Adding pileup to ' + hard_filename + ' with pileup mean', n_expected)

    with h5py.File(hard_filename, 'r') as input_file:
        input_events = input_file['events']
        n_events_total = len(input_events)

        with h5py.File(output_filename, 'w') as output_file:
            dt_momentum = np.dtype([('E', np.float16), ('px', np.float16), ('py', np.float16), ('pz', np.float16)])
            #dt_event = h5py.special_dtype(vlen='float32, float32, float32, float32')
            dt_event = h5py.special_dtype(vlen=dt_momentum)
            output_events = output_file.create_dataset('events', (n_events_total,), dtype=dt_event)

            total_hard_particles = 0
            total_pileup_particles = 0
            n_events = 0

            for event in input_events:
                
                # Get hard momenta
                n_momenta = len(event)
                momenta = np.zeros((n_momenta,), dtype = dt_momentum)
                for i, p in enumerate(event):
                    momenta[i] = (p[0], p[1], p[2], p[3])
                total_hard_particles += n_momenta

                # Calculate amount of pileup
                if use_poisson:
                    n_pileup = poisson(n_expected)
                else:
                    n_pileup = n_expected

                # Get pileup momenta
                pileup_momenta_raw = []
                for i in range(n_pileup):
                    try:
                        pileup_momenta_raw += list(pickle.load(pileup_file, encoding='latin1'))
                    except EOFError:
                        # At end of file, rewind
                        pileup_file.seek(0)
                        pileup_momenta_raw += list(pickle.load(pileup_file, encoding='latin1'))
                        print('Starting from beginning of pileup file.')
                n_pileup_particles = len(pileup_momenta_raw)
                pileup_momenta = np.zeros((n_pileup_particles,), dtype = dt_momentum)
                for i, p in enumerate(pileup_momenta_raw):
                    pileup_momenta[i] = (p[3], p[0], p[1], p[2])
                total_pileup_particles += n_pileup_particles

                if len(pileup_momenta) > 0:
                    momenta = np.hstack((momenta, pileup_momenta))
                    
                output_events[n_events] = momenta
                n_events += 1

                if n_events % 1000 == 0:
                    print(' ', n_events, 'events done, mean particles:',
                          total_hard_particles / n_events, '+', total_pileup_particles / n_events)

    print('Summary:')
    print('  Events:', n_events)
    print('  Average hard particles:', total_hard_particles / n_events)
    print('  Average pileup particles:', total_pileup_particles / n_events)

In [12]:
# Go!
folder = '/scratch/jb6504/data'
#folder = '../data'
hard_filenames = [folder + '/w-vs-qcd/h5/w_100000.h5',
                  folder + '/w-vs-qcd/h5/qcd_100000.h5']
n_expected = [25,
              25]
output_filenames = [folder + '/w-vs-qcd/h5/w_100000_pileup25.h5',
                    folder + '/w-vs-qcd/h5/qcd_100000_pileup25.h5']
pileup_filename = folder + '/pileup/pileup.dat.tar.bz2'

pileup_file = bz2.BZ2File(pileup_filename, "r")

for hard_filename, n, output_filename in zip(hard_filenames, n_expected, output_filenames):
    add_pileup_jet_particle(hard_filename, output_filename, pileup_file, n, True)


Adding pileup to /scratch/jb6504/data/w-vs-qcd/qcd_100000.h5 with pileup mean 25
  1000 events done, mean particles: 537.022 + 4704.372
  2000 events done, mean particles: 536.911 + 4724.8675
  3000 events done, mean particles: 531.5406666666667 + 4729.078666666666
  4000 events done, mean particles: 531.73625 + 4721.10825
  5000 events done, mean particles: 531.8486 + 4718.4184
  6000 events done, mean particles: 533.4206666666666 + 4711.868
  7000 events done, mean particles: 533.673 + 4709.3198571428575
  8000 events done, mean particles: 534.334125 + 4711.255375
  9000 events done, mean particles: 535.0223333333333 + 4714.552222222223
  10000 events done, mean particles: 533.8433 + 4718.1569
  11000 events done, mean particles: 534.7271818181819 + 4715.762727272728
  12000 events done, mean particles: 533.8863333333334 + 4714.04175
  13000 events done, mean particles: 533.7266153846153 + 4712.987
  14000 events done, mean particles: 534.3152857142857 + 4710.669
  15000 events done

# Full event, particle level

In [13]:
def add_pileup_event_particle(hard_filename, output_filename,
                              pileup_file, n_expected, use_poisson=True):
    
    ''' Adds pileup to a particle-level event file. '''

    print('')
    print('Adding pileup to ' + hard_filename + ' with pileup mean', n_expected)

    with open(hard_filename, 'rb') as input_file:
    #with bz2.BZ2File(hard_filename, "r") as input_file:
        with bz2.BZ2File(output_filename, "w") as output_file:
            
            total_hard_particles = 0
            total_pileup_particles = 0
            n_events = 0

            while True:
                try:
                    momenta = pickle.load(input_file, encoding='latin1')
                except EOFError:
                    break
                n_events += 1
                total_hard_particles += len(momenta)
                
                if use_poisson:
                    n_pileup = poisson(n_expected)
                else:
                    n_pileup = n_expected
                
                pileup_momenta = []
                for i in range(n_pileup):
                    try:
                        pileup_momenta += list(pickle.load(pileup_file, encoding='latin1'))
                    except EOFError:
                        # At end of file, rewind
                        pileup_file.seek(0)
                        pileup_momenta += list(pickle.load(pileup_file, encoding='latin1'))
                        print('Starting from beginning of pileup file.')
                total_pileup_particles += len(pileup_momenta)
                pileup_momenta = np.asarray(pileup_momenta)
                
                if len(pileup_momenta) > 0:
                    momenta = np.vstack((momenta, pileup_momenta))
                pickle.dump(momenta, output_file)

                if n_events % 10000 == 0:
                    print(' ', n_events, 'events done, mean particles:',
                          total_hard_particles / n_events, '+', total_pileup_particles / n_events)

    print('Summary:')
    print('  Events:', n_events)
    print('  Average hard particles:', total_hard_particles / n_events)
    print('  Average pileup particles:', total_pileup_particles / n_events)

In [14]:
# Go!
folder = '/scratch/jb6504/data'
#folder = '../data'
hard_filenames = [folder + '/events/wprime-particles.dat',
                  folder + '/events/dijet-particles.dat']
n_expected = [25,
              25]
output_filenames = [folder + '/events/wprime-particles-pileup25.dat.bz2',
                    folder + '/events/dijet-particles-pileup25.dat.bz2']
pileup_filename = folder + '/pileup/pileup.dat.bz2'

#pileup_file = open(pileup_filename, 'rb') 
pileup_file = bz2.BZ2File(pileup_filename, "r")

for hard_filename, n, output_filename in zip(hard_filenames, n_expected, output_filenames):
    add_pileup_event_particle(hard_filename, output_filename, pileup_file, n, True)


Adding pileup to /scratch/jb6504/data/events/dijet-particles.dat with pileup mean 25
  10000 events done, mean particles: 530.4376 + 4699.4621
  20000 events done, mean particles: 531.0632 + 4694.9929
  30000 events done, mean particles: 530.9473333333333 + 4699.0671
  40000 events done, mean particles: 531.29085 + 4692.46765
  50000 events done, mean particles: 531.02312 + 4696.99316
  60000 events done, mean particles: 530.807 + 4699.600416666667
  70000 events done, mean particles: 530.9862857142857 + 4699.089557142857
Starting from beginning of pileup file.
  80000 events done, mean particles: 530.9101375 + 4702.6208375
  90000 events done, mean particles: 530.8895 + 4705.0044333333335
  100000 events done, mean particles: 530.96907 + 4704.84843
  110000 events done, mean particles: 531.0724181818182 + 4704.951145454545
  120000 events done, mean particles: 530.7914166666667 + 4704.724333333334
  130000 events done, mean particles: 530.7209769230769 + 4705.773946153846
  140000 ev